In [21]:
import pandas as pd
from forecast import predict, load_data, sliding_windows_predict
from tsai.all import load_model, TCN
from pathlib import Path
import torch

if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [22]:
sc, data, _, _, _, _ = load_data(
    base_path=Path('./data/dfyj'), ip='84.123.58.224', rolling=30, device=device)

dataX_predict = torch.from_numpy(sliding_windows_predict(data))
dataX_predict = dataX_predict.reshape(
    dataX_predict.shape[0], 1, dataX_predict.shape[1])

load_model('./models/84.123*.pth', TCN)

predictions = predict(dataX_predict, learner, sc)

AttributeError: 'collections.OrderedDict' object has no attribute 'get_X_preds'